In [69]:
import json
from tqdm import tqdm
# read json from data/json/tags.json and band.json
def remove_longest_string(lst):
    if not lst:
        return lst  # Return the list as is if it's empty
    
    # Find the longest string
    longest_string = max(lst, key=len)
    
    # Remove the longest string
    lst.remove(longest_string)
    
    return lst

def filter_keywords(keyword_list):
    filtered_keywords = []
    for keyword in keyword_list:
        if not keyword.isdigit():  # Check if the keyword is not only numbers
            if all(ord(char) < 128 for char in keyword):  # Check if keyword is English
                if len(keyword) >= 3:
                    filtered_keywords.append(keyword)
            else:
                filtered_keywords.append(keyword)
    return filtered_keywords


with open('data/json/tags.json', 'r') as f:
    tags = json.load(f)

tmp = []
for tag in tags:
    if len(tmp)>=2:
        tmp.append(tag['name'])
tags = tmp


In [70]:

tmp = []
with open('data/json/band.json', 'r') as f:
    band = json.load(f)
    for b in band:
        band_keyword = b['band_keyword'] 
        band_keyword.sort(key=len, reverse=True)
        band_keyword = band_keyword[:3]  # 只取前三個關鍵字

        #If an element in band_keyword is in English,  ensure that the element must have a length of at least 3 characters
        band_keyword = [elm if not (elm.isalpha() and len(elm) < 3) else None for elm in band_keyword]
        band_keyword = [elm for elm in band_keyword if elm is not None]
        print(band_keyword)
        tmp.extend(band_keyword)
band = tmp

['lancôme蘭蔻', 'lancome', 'lancôme']
['chanel香奈兒', '香奈兒']
['dior迪奧', 'dior']
['shiseido資生堂國際櫃']
['whoo后', 'whoo']
['estée lauder雅詩蘭黛', 'estee lauder', 'estée lauder']
["kiehl's契爾氏", "kiehl's", 'kiehls']
["l'oreal paris巴黎萊雅", "l'oreal paris", 'loreal paris']
['neogence霓淨思', '霓淨思']
['m.a.c', 'mac']
['kanebo佳麗寶-專櫃', '佳麗寶']
['kose高絲-專櫃']
['資生堂東京櫃', '資生堂']
['clé de peau beauté肌膚之鑰', 'cle de peau beaute', 'clé de peau beauté']
['dr.wu達爾膚', 'dr.wu', 'drwu']
['hada-labo肌研', 'hadalabo']
['sk-ii', 'skii']
['clarins克蘭詩', 'clarins', '克蘭詩']
['la roche-posay理膚寶水', 'la rocheposay', 'larocheposay']
['giorgio armani beauty亞曼尼', 'giorgio armani beauty', 'giorgioarmanibeauty']
['dhc']
['sofina蘇菲娜', 'sofina', '蘇菲娜']
['maybelline媚比琳', 'maybelline', '媚比琳']
['mentholatum曼秀雷敦', 'mentholatum', '曼秀雷敦']
['bobbi brown芭比波朗', 'bobbi brown', 'bobbibrown']
['ysl聖羅蘭', '聖羅蘭', 'ysl']
['guerlain嬌蘭', 'guerlain']
['etude']
['innisfree']
['orbis']
['laneige蘭芝']
['shu uemura植村秀', 'shu uemura', 'shuuemura']
['bio-essence碧歐斯', 

In [71]:
# save tag as flat list
with open('data/json/tags_flat.json', 'w') as f:
    json.dump(tags, f, ensure_ascii=False, indent=4)    

In [72]:
len(tags)

0

In [73]:

# Step 1: Extraction

with open('data/json/makeup.json', 'r') as f:
    makeup = json.load(f)




In [74]:
import re
def merge_push_content_by_userid(messages):
    if not isinstance(messages, list):
        raise ValueError("The input messages must be a list.")
    
    result = {}
    for message in messages:
        userid = message["push_userid"]
        content = message["push_content"]
        if userid in result:
            result[userid].append(content)
        else:
            result[userid] = [content]
    output = []
    for k,v in result.items():
        output.append(v)
    return output

def is_review_post(post_title):
    return post_title.startswith("[心得]")

def find_elements_in_string(input_string, elements_list):
    # Convert the string to lower case to make the search case-insensitive
    input_string = input_string.lower()

    # Initialize an empty list to store the elements found in the string
    found_elements = []

    # Iterate through each element in the elements list
    for element in elements_list:
        # Check if the element is present in the string
        if element.lower() in input_string:
            # If present, add the element to the found_elements list
            found_elements.append(element)

    # Return the list of found elements
    return found_elements

def remove_punctuation(input_string):
    # Define a regular expression pattern to match all punctuation marks
    pattern = r'[^\w\s]'
    # Use re.sub to replace all punctuation marks with an empty string
    cleaned_string = re.sub(pattern, '', input_string)
    return cleaned_string

def remove_links(text):
    # Regular expression pattern to match URLs
    url_pattern = r'https?://\S+|www\.\S+'
    # Substitute matched URLs with an empty string
    cleaned_text = re.sub(url_pattern, '', text)
    return cleaned_text
# Step 2: Transformation
posts = []


query_tag = "防曬"

'''
'''
for i,_ in enumerate(makeup):
    if query_tag not in _['content']:
        # print(f"Skip post {i} because it does not contain the query tag '{query_tag}'")
        continue
    if _['article_title'] and not is_review_post(_['article_title']) :
        continue
    post ={}
    post['title'] = _['article_title']
    post['tags'] = find_elements_in_string(_['content'], tags)
    post['band'] = find_elements_in_string(_['content'], band)
    post['content'] = remove_punctuation(remove_links(_['content']))
    messages = merge_push_content_by_userid(_['messages'])
    messages  = [remove_punctuation(remove_links(''.join(m))) for m in messages]
    messages = [m for m in messages if len(m)>2]   # filter out message with length less than 2
    post['messages'] = messages

    posts.append(post)

    


In [75]:
def split_string_fixed_size(input_string, size, delimiter=" "):
    """
    Splits the input string into a list of strings each of fixed size,
    adding the previous and next segments for context.

    Args:
    input_string (str): The string to be split.
    size (int): The fixed size for each split part.
    delimiter (str): The delimiter used to split the input string into segments.

    Returns:
    list: A list of strings split by the fixed size, with added context.
    """
    parts = []
    segments = input_string.split(delimiter)  # 根据传入的分隔符分割字符串
    
    for i in range(len(segments)):
        current_segment = segments[i]
        
        if i > 0:
            prev_segment = segments[i-1]
        else:
            prev_segment = ""
        
        if i < len(segments) - 1:
            next_segment = segments[i+1]
        else:
            next_segment = ""
        
        context_segment = prev_segment + delimiter + current_segment + delimiter + next_segment
        parts.extend([context_segment[j:j + size] for j in range(0, len(context_segment), size)])
    
    return parts

# Example usage in your loop
posts_chunk = []
fixed_content_size = 250
for p in posts:
    title = p['title'].replace('[心得]', '心得:') if p['title'] else ''
    user_review = '使用者評價:'
    band = '品牌:' + ' '.join(p['band']) + '\n'

    for message in p['messages']:
        user_review += f";{''.join(message)}"

    user_review_chunks = split_string_fixed_size(user_review, fixed_content_size, delimiter=" ")
    for u in user_review_chunks:
        posts_chunk.append(title + '\n' + u + '\n' + band)


In [76]:
for i in range(20):
    print(posts_chunk[i])
    print()

心得: Dr.Althea 女神水光氣墊粉餅使用心得
 使用者評價:;眼睛好美喲;剛完妝那張真是美翻了;盒子好漂亮;我覺得浮粉蠻嚴重的lol 不是會和出油融合的感覺;近照感覺不太ok;近照全臉都浮粉了吧
品牌:sofina you


心得: Dr.Althea 女神水光氣墊粉餅使用心得
使用者評價:;眼睛好美喲;剛完妝那張真是美翻了;盒子好漂亮;我覺得浮粉蠻嚴重的lol 不是會和出油融合的感覺;近照感覺不太ok;近照全臉都浮粉了吧 不過剛上很美
品牌:sofina you


心得: Dr.Althea 女神水光氣墊粉餅使用心得
不是會和出油融合的感覺;近照感覺不太ok;近照全臉都浮粉了吧 不過剛上很美 
品牌:sofina you


心得: 雅詩霧感唇膏#330-搶眼實搭的橘紅！
 使用者評價:;這反差有點厲害顏色好看;這次的霧感唇膏真的超美週年慶想買個兩支;顏色好美;顏色適合妳;美呆了;顏色很適合你唇嘟嘟的擦得很好看;好看耶被雅詩這一系列生火了XD覺得Vivian代言真的超級超級燒;這顏色好美喔很遮妳的唇色我覺得這系列霧面的都不乾很好擦;妳害我抑制半年多的唇膏慾又浮現了_;顏色真的很美好看好看;顏色好美;原PO這顏色好好看耶 感覺秋冬就是要來一隻這種楓葉紅唇膏;顏色太美了也想去靠櫃試試看;這顏色很美很適合你;顏色很漂亮擦起來很美;好漂亮很適合喔
品牌:


心得: 雅詩霧感唇膏#330-搶眼實搭的橘紅！
使用者評價:;這反差有點厲害顏色好看;這次的霧感唇膏真的超美週年慶想買個兩支;顏色好美;顏色適合妳;美呆了;顏色很適合你唇嘟嘟的擦得很好看;好看耶被雅詩這一系列生火了XD覺得Vivian代言真的超級超級燒;這顏色好美喔很遮妳的唇色我覺得這系列霧面的都不乾很好擦;妳害我抑制半年多的唇膏慾又浮現了_;顏色真的很美好看好看;顏色好美;原PO這顏色好好看耶 感覺秋冬就是要來一隻這種楓葉紅唇膏;顏色太美了也想去靠櫃試試看;這顏色很美很適合你;顏色很漂亮擦起來很美;好漂亮很適合喔 
品牌:


心得: 東京購入CANMAKE HAUL-多樣新品心得
 使用者評價:;好滿足;我覺得貝殼腮紅霜滿雷的 顯色度很差;看的超開心我也很喜歡Canmake;我代購買的腮紅刷沒有刷具袋QQ;有canmake推貝殼腮紅也推推擦起來蠻漂亮的;那個眉粉個人也覺得很難用;最近剛從沖繩回來
品牌

In [77]:
# Step 3: Load save posts_chunk to file
with open('data/json/posts_chunk_防曬.json', 'w') as f:
    json.dump(posts_chunk, f, ensure_ascii=False, indent=4)


In [78]:
from typing import AsyncIterator, Iterator

from langchain_core.document_loaders import BaseLoader
from langchain_core.documents import Document


class PTTDocumentLoader(BaseLoader):
    def __init__(self, file_path: str) -> None:
        self.file_path = file_path

    def lazy_load(self) -> Iterator[Document]:  # <-- Does not take any arguments

        with open(self.file_path, 'r') as f:
            makeup = json.load(f)

            line_number = 0
            for line in makeup:
                yield Document(
                    page_content=line,
                    metadata={"line_number": line_number, "source": self.file_path},
                )
                line_number += 1

    # alazy_load is OPTIONAL. If you leave out the implementation, a default implementation which delegates to lazy_load will be used!
    async def alazy_load(self) -> AsyncIterator[Document]:  # <-- Does not take any arguments
        async with aiofiles.open(self.file_path, 'r', encoding='utf-8') as f:
            contents = await f.read()
            makeup = json.loads(contents)
    
            line_number = 0
            for line in makeup:
                yield Document(
                    page_content=line,
                    metadata={"line_number": line_number, "source": self.file_path},
                )
                line_number += 1